### Properties

To be clear, here we are examining **instance** properties. That is, we define the property in the class we are defining, but the property itself is going to be **instance** specific, i.e. different instances will support different values for the property. Just like instance attributes. The main difference is that we will use accessor method to get, set (and optionally) delete the associated instance value.

As I mentioned in the lecture, because properties use the same dotted notation (and the same `getattr`, `setattr` and `delattr` functions), we do not need to **start** with properties. Often a bare attribute works just fine, and if, later, we decide we need to manage getting/setting/deleting the attribute value, we can switch over to properties without breaking our class interface. This is unlike languages like Java - and hence why in those languages it is recommended to **always** use getter and setter functions. *Not so* in Python!

A **property** in Python is essentially a class instance - we'll come back to what that class looks like when we study descriptors. For now, we are going to use the `property` function in Python which is a convenience callable essentially.

Let's start with a simple example and a bare attribute:

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name

So this class has a single instance **attribute**, `name`.

In [ ]:
p = Person('Alex')

And we can access and modify that attribute using either dotted notation or the `getattr` and `setattr` methods:

In [ ]:
p.name

In [ ]:
getattr(p, 'name')

p.name = 'John'

In [ ]:
p.name

In [ ]:
setattr(p, 'name', 'Eric')

In [ ]:
p.name

Now suppose we wan't to disallow setting an empty string or `None` for the name. Also, we'll require the name to be a string.

To do that we are going to create an instance method that will handle the logic and setting of the value. We also create an instance method to retrieve the attribute value.

We'll use `_name` as the instance variable to store the name.

In [ ]:
class Person:
    def __init__(self, name):
        self.set_name(name)
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')

In [ ]:
p = Person('Alex')

In [ ]:
try:
    p.set_name(100)
except ValueError as ex:
    print(ex)

In [ ]:
p.set_name('Eric')

In [ ]:
p.get_name()

Of course, our users can still manipulate the atribute directly if they want by using the "private" attribute `_name`. You can't stop anyone from doing this in Python - they should know better than to do that, but we're all good programmers, and know what and what not to do, hopefully!

The way we set up our initializer, the validation will work too:

In [ ]:
try:
    p = Person('')
except ValueError as ex:
    print(ex)

So this works, but it's a bit of pain to use the method names. So let's turn this into a property instead. We start with the class we just had and tweak it a bit:

In [ ]:
class Person:
    def __init__(self, name):
        self.name = name  # note how we are actually setting the value for name using the property!
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        if isinstance(value, str) and len(value.strip()) > 0:
            # this is valid
            self._name = value.strip()
        else:
            raise ValueError('name must be a non-empty string')
            
    name = property(fget=get_name, fset=set_name)

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.name = 'Eric'

In [ ]:
try:
    p.name = None
except ValueError as ex:
    print(ex)

So now we have the benefit of using accessor methods, without having to call the methods explicitly.

In fact, even `getattr` and `setattr` will work the same way:

In [ ]:
setattr(p, 'name', 'John')  # or p.name = 'John'

In [ ]:
getattr(p, 'name')  # or simply p.name

Now let's examine the instance dictionary:

In [ ]:
p.__dict__

You'll see we can find the underlying "private" attribute we are using to store the name. But the property itself (`name`) is not in the dictionary.

The property was defined in the class:

In [ ]:
Person.__dict__

And you can see it's type is `property`.

So when we type `p.name` or `p.name = value`, Python recognizes that `'name` is a `property` and therefore uses the accessor methods. (How it does we'll see later when we study descriptors).

What's interesting is that even if we muck around with the instance dictionary, Python does not get confused - (and as usual in Python, just because you **can** do something does not mean you **should**!)

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.__dict__

In [ ]:
p.__dict__['name'] = 'John'

In [ ]:
p.__dict__

As you can see, we now have `name` in our instance dictionary.

Let's retrieve the `name` via dotted notation:

In [ ]:
p.name

That's obviously still using the getter method.

And setting the name:

In [ ]:
p.name = 'Raymond'

In [ ]:
p.__dict__

As you can see, it used the setter method.

And the same thing happens with `setattr` and `getattr`:

In [ ]:
getattr(p, 'name')

In [ ]:
setattr(p, 'name', 'Python')

In [ ]:
p.__dict__

As you can see the `setattr` method used the property setter.

For completeness, let's see how the deleter method works:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        print('getting name')
        return self._name
    
    def set_name(self, value):
        print('setting name')
        self._name = value
        
    def del_name(self):
        print('deleting name')
        del self._name  # or whatever "cleanup" we want to do
        
    name = property(get_name, set_name, del_name)

In [ ]:
p = Person('Alex')

In [ ]:
p.__dict__

In [ ]:
p.name

In [ ]:
p.name = 'Eric'

In [ ]:
p.__dict__

In [ ]:
del p.name

In [ ]:
p.__dict__

Now, the property still exists (since it is defined in the class) - all we did was remove the underlying value for the property (the `_name` instance attribute):

In [ ]:
try:
    p.name
except AttributeError as ex:
    print(ex)

As you can see the issue is that the getter function is trying to find `_name` in the attribute, which no longer exists. So the getter and setter still exist (i.e. the property still exists), so we can still assign to it:

In [ ]:
p.name = 'Alex'

In [ ]:
p.name

The last param in `property` is just a docstring. So we could do this:

In [ ]:
class Person:
    """This is a Person object"""
    def __init__(self, name):
        self._name = name
        
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
        
    name = property(get_name, set_name, doc="The person's name.")

In [ ]:
p = Person('Alex')

In [ ]:
help(Person.name)

In [ ]:
help(Person)


`The easy way !`

Hopefully you can now see where the original property (with just the getter), had a callable `setter` that "added" the setter to the property (by creating a new property with both getter and setter), that also returned the (new) property object.

So, we can simplify our code this way:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # what's the property name now? --> name
    # so name has a setter callable
    @name.setter
    def name(self, value):
        self._name = value

Note that if we had not named our setter function `name` the property name would have changed!

Remember that:
```
@dec
def my_func():
    pass
 ```
 returns a decorated function with the same name as the original function

In [ ]:
Person.__dict__

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.name = 'Guido'
p.name

Just to show you, if we had not used the same name for the setter function:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    # property is now called name
    
    @name.setter
    def full_name(self, value):
        self._name = value

In [ ]:
Person.__dict__

As you can see we now have two properties on the class! The first one `name` will only work as a getter. And the second one `full_name` will work as both a getter and a setter:

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.full_name

In [ ]:
p.full_name = 'Raymond'

In [ ]:
p.full_name

But this won't work:

In [ ]:
try:
    p.name = 'Guido'
except AttributeError as ex:
    print(ex)

Technically, the property callable has both a getter and setter method - so we can create the setter first, then "add in" the getter. But since the first argument to `property` is the getter, we have to work a bit more to do it:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value

By the way, we now have a property that can be set, but not read back!

In [ ]:
p = Person('Alex')

In [ ]:
p.__dict__

In [ ]:
p.name = 'Raymond'

In [ ]:
p.__dict__

In [ ]:
try:
    p.name
except AttributeError as ex:
    print(ex)

So, if you ever need an attribute that is "write-only" - you can do it. Maybe the data is sensitive, and you want to set it, but not show back to users... But the data is never truly private, so at best you're obfuscating the data - so in my experience I've never had to do something like that. Just wanted you to see this in case the need ever came up.

But let's finish this up and make the property read/write:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property()  # an "empty" prroperty - no getter or setter
    
    @name.setter
    def name(self, value):
        self._name = value
        
    @name.getter
    def name(self):
        return self._name

In [ ]:
p = Person('Alex')

In [ ]:
p.name

In [ ]:
p.name = 'Raymond'

In [ ]:
p.name

The deleter works the same way, and we'll come back to it soon.

Lastly you'll recall that we could set up a docstring when using the `property` callable.

The standard technique is to simply define the docstring in the getter function:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        """The Person's name."""
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

In [ ]:
help(Person.name)

In [ ]:
help(Person)

What happens if we set it in the setter instead?

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        """The Person's name."""
        self._name = value

In [ ]:
help(Person.name)

In [ ]:
help(Person)

As you can see, the property docstring is only set on the getter. So how to set a docstring with a write-only property? We can do that when we create the initial property:

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name
        
    name = property(doc='Write-only name property.')
    
    @name.setter
    def name(self, value):
        self._name = value

In [ ]:
help(Person.name)